#### 딥러닝 MLP

In [ ]:
import joblib
import pandas as pd
import torch
from train import train_model

# 1. 입력 특성 수 계산
X_train = pd.read_csv("../data/processed/train.csv").drop(columns=["Rented_Bike_Count"])
input_dim = X_train.shape[1]

# 2. 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3. 모델 학습 + 스케일러 저장됨
model = train_model(
    train_path="../data/processed/train.csv",
    val_path="../data/processed/val.csv",
    input_dim=input_dim,
    epochs=200,
    patience=10,
    device=device
)

# 4. 저장된 스케일러 로드
scaler = joblib.load("../models/input_scaler.pkl")

# 5. 테스트셋 로드 및 정규화
df_test = pd.read_csv("../data/processed/test.csv")
X_test = scaler.transform(df_test.drop(columns=["Rented_Bike_Count"]).values)
y_test = df_test["Rented_Bike_Count"].values.reshape(-1, 1)

# 6. 평가 (정규화된 X_test 사용)
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    X_tensor = torch.tensor(X_test.astype(np.float32)).to(device)
    preds = model(X_tensor).cpu().numpy()

# 7. 평가 결과 출력 및 저장
results.append(print_mse("MLP (PyTorch)", mean_squared_error(y_test, preds)))

# 8. 시각화
plt.figure(figsize=(8, 6))
plt.scatter(y_test, preds, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("True Count")
plt.ylabel("Predicted Count")
plt.title("True vs Predicted (Scaled Input)")
plt.show()